# Notebook 3 — Joins, Subqueries & CTEs (Artists on MySQL)

_INFO 4614 • Unit 1: Relational Databases_

**This notebook assumes a MySQL database on the course RDS instance.** The schema mirrors the sample `artist_term.db` you saw in Python, now translated to SQL on MySQL.

**Tables (expected):**
- `artists(artist_id PK)`
- `terms(term PK)`
- `mbtags(mbtag PK)`
- `artist_term(artist_id, term)` — maps artists ↔︎ terms
- `artist_mbtag(artist_id, mbtag)` — maps artists ↔︎ MusicBrainz-style tags

**What you'll practice (last guided notebook before the midterm):**
- Core joins: **INNER** and **LEFT** (multi‑table queries)
- Translating **research questions → SQL** (reduced scaffolding)
- **Subqueries** (`IN`, `EXISTS`) and **CTEs** (`WITH`) for clarity
- Set ops: `UNION` vs `UNION ALL`

**Submission checklist**

- ✅ All cells run top–to–bottom without errors
- ✅ You completed both the **scaffolded** and **independent** join tasks
- ✅ You wrote at least **two subqueries** (one `IN`/`EXISTS`) and **one CTE**
- ✅ You added brief interpretations where prompted
- ✅ File is renamed `Notebook3_LastFirst.ipynb` before you submit

## 0) Setup & connection (MySQL on RDS)

We’ll reuse the connection pattern from **Notebook 1** (TLS required). If the database name differs from the default below, **update `DB_NAME`**.

**Quoting identifiers.** If a column has spaces/punctuation, use backticks in MySQL (e.g., ``SELECT `weird name` FROM t``).

In [ ]:
# Minimal dependencies
!pip -q install mysql-connector-python SQLAlchemy pandas matplotlib

import sys, sqlalchemy, pandas
print("Python:", sys.version.split()[0])
print("SQLAlchemy:", sqlalchemy.__version__)
print("pandas:", pandas.__version__)

In [ ]:
# --- RDS Connection Settings (update DB_NAME if your instructor chose a different name) ---
DB_HOST = "info4614.c7kemoi0y6yq.us-east-2.rds.amazonaws.com"
DB_PORT = 3306
DB_NAME = "artists"   # ← change here if the course DB name differs
CA_CERT_PATH = "global-bundle.pem"

In [ ]:
# Download the Amazon RDS global CA bundle (TLS)
!wget -q https://truststore.pki.rds.amazonaws.com/global/global-bundle.pem -O "$CA_CERT_PATH"

import os
assert os.path.exists(CA_CERT_PATH), "PEM download failed — try again or upload manually"
print("Saved:", os.path.abspath(CA_CERT_PATH))

In [ ]:
# Alternative: upload a PEM you already have
try:
    from google.colab import files
    print("Upload global-bundle.pem if needed")
    uploaded = files.upload()
except Exception as e:
    print("Upload not available here:", e)

### Enter your read‑only credentials

In [ ]:
from getpass import getpass
DB_USER = input("MySQL username: ").strip()
DB_PASS = getpass("MySQL password: ").strip()

### Create SQLAlchemy engine (MySQL) and sanity check

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

engine = create_engine(
    f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}",
    connect_args={"ssl_ca": CA_CERT_PATH},
    pool_pre_ping=True,
)

with engine.connect() as conn:
    print("SELECT 1 →", conn.execute(text("SELECT 1")).scalar_one())
    cipher = conn.execute(text("SHOW SESSION STATUS LIKE 'Ssl_cipher'")).fetchone()
    print("SSL cipher:", cipher[1] if cipher else None)

## 1) Quick schema peek

In [ ]:
import pandas as pd
from sqlalchemy import text

with engine.connect() as conn:
    print("Tables:")
    display(pd.read_sql_query(text("SHOW TABLES"), conn))

    for t in ("artists","terms","mbtags","artist_term","artist_mbtag"):
        try:
            print(f"\nDESCRIBE {t}:")
            display(pd.read_sql_query(text(f"DESCRIBE {t}"), conn))
        except Exception as e:
            pass

## From research question → SQL query (quick recipe)

1) **What rows?** → `FROM ... WHERE ...`  
2) **Unit of analysis?** → choose grouping keys or join keys  
3) **Which tables?** → link tables (`artist_term`, `artist_mbtag`) and dimensions (`terms`, `mbtags`, `artists`)  
4) **How to connect them?** → `JOIN ... ON ...`  
5) **What to measure?** → `COUNT(*)`, `COUNT(DISTINCT ...)`, `MIN/MAX/AVG`  
6) **Which groups qualify?** → `HAVING` (after aggregation)  
7) **Present it well.** → `ORDER BY`, `LIMIT`, clean column aliases

## 2) Sanity checks

Let’s count rows in each table to understand scale.

In [ ]:
import pandas as pd
from sqlalchemy import text

with engine.connect() as conn:
    for q in [
        "SELECT COUNT(*) AS n FROM artists",
        "SELECT COUNT(*) AS n FROM terms",
        "SELECT COUNT(*) AS n FROM mbtags",
        "SELECT COUNT(*) AS n FROM artist_term",
        "SELECT COUNT(*) AS n FROM artist_mbtag",
    ]:
        print(q)
        display(pd.read_sql_query(text(q), conn))

## 3) Joins via research questions

Two **scaffolded** join questions to warm up, then two **independent** ones.

### RQ 1 — Which **terms** are attached to the most **artists**?
Return the **top 15 terms** by the number of distinct artists using them.

**Translate → SQL (hint).**
- Tables: `artist_term` (links) → `terms` (labels)
- Join key: `artist_term.term = terms.term`
- Unit of analysis: each `terms.term`
- Measure: `COUNT(DISTINCT artist_term.artist_id) AS n`
- Present: `ORDER BY n DESC LIMIT 15`

In [ ]:
# 🔧 Scaffolded JOIN — fill the blanks
import pandas as pd
from sqlalchemy import text

sql = """
SELECT
  t.term,
  COUNT(DISTINCT at.artist_id) AS n
FROM artist_term AS at
JOIN terms AS t
  ON at.term = t.term
GROUP BY ...
ORDER BY ...
LIMIT ...
"""
with engine.connect() as conn:
    df_top_terms = pd.read_sql_query(text(sql), conn)

df_top_terms.head(15)

### RQ 2 — Which **artists** have the most **MusicBrainz tags**?
Return the top 15 artists by **count of distinct `mbtag`**.

**Translate → SQL (hint).**
- Tables: `artist_mbtag` (links) → `artists` (IDs/names if present)
- Join key: `artist_mbtag.artist_id = artists.artist_id`
- Unit of analysis: each `artists.artist_id`
- Measure: `COUNT(DISTINCT artist_mbtag.mbtag) AS n`
- Present: `ORDER BY n DESC LIMIT 15`

In [ ]:
# 🔧 Scaffolded JOIN — fill the blanks
sql = """
SELECT
  a.artist_id,
  COUNT(DISTINCT am.mbtag) AS n
FROM artist_mbtag AS am
JOIN artists AS a
  ON ...
GROUP BY ...
ORDER BY ...
LIMIT ...
"""
with engine.connect() as conn:
    df_top_taggy = pd.read_sql_query(text(sql), conn)

df_top_taggy.head(15)

### RQ 3 (independent) — What are the **top 10 mbtags** overall?
Return two columns: `mbtag`, `n` (number of distinct artists using that tag), sorted by `n` desc, limited to 10.

> Hints (only if needed): group by `mbtag`; you may only need the link table + `mbtags` for labels.

In [ ]:
# 🔎 Your turn — write the whole SQL
sql = """
-- write your query here
"""
with engine.connect() as conn:
    df_top_mbtags = pd.read_sql_query(sql, conn)

df_top_mbtags

### RQ 4 (independent) — Among artists tagged with the **term 'rock'**, which **mbtags** are most common?
Return `mbtag`, `n` (distinct artists) for the **top 10** mbtags among artists that have the **term 'rock'` (case-insensitive is fine).

> Hint (if needed): join `artist_term` ↔ `artist_mbtag` via `artist_id`. Filter rows first, then aggregate.

In [ ]:
# 🔎 Your turn — write the whole SQL
sql = """
-- write your query here
"""
with engine.connect() as conn:
    df_rock_mbtags = pd.read_sql_query(sql, conn)

df_rock_mbtags.head(10)

## 4) Subqueries (IN / EXISTS) and CTEs

We’ll express the same question two ways: once as a **subquery** and once as a **CTE**. MySQL 8+ supports `WITH` (CTEs).

### RQ 5 — Which artists have **both** the term `'rock'` **and** the mbtag `'seen live'`?
Return the first 20 `artist_id` values that satisfy both conditions, sorted ascending.

- **Option A (subqueries):** use `IN` or `EXISTS` twice.
- **Option B (CTE):** build two CTEs (`rock_artists`, `seenlive_artists`) then `INNER JOIN` them.

In [ ]:
# ▶️ Option A — Subqueries
from sqlalchemy import text
sql = """
SELECT a.artist_id
FROM artists AS a
WHERE a.artist_id IN (
  SELECT at.artist_id
  FROM artist_term AS at
  WHERE LOWER(at.term) = 'rock'
)
AND a.artist_id IN (
  SELECT am.artist_id
  FROM artist_mbtag AS am
  WHERE LOWER(am.mbtag) = 'seen live'
)
ORDER BY a.artist_id
LIMIT 20;
"""
with engine.connect() as conn:
    df_both_subq = pd.read_sql_query(text(sql), conn)

df_both_subq

In [ ]:
# ▶️ Option B — CTEs
from sqlalchemy import text
sql = """
WITH rock_artists AS (
  SELECT DISTINCT artist_id
  FROM artist_term
  WHERE LOWER(term) = 'rock'
),
seenlive_artists AS (
  SELECT DISTINCT artist_id
  FROM artist_mbtag
  WHERE LOWER(mbtag) = 'seen live'
)
SELECT ra.artist_id
FROM rock_artists AS ra
JOIN seenlive_artists AS sa
  ON ra.artist_id = sa.artist_id
ORDER BY ra.artist_id
LIMIT 20;
"""
with engine.connect() as conn:
    df_both_cte = pd.read_sql_query(text(sql), conn)

df_both_cte

### RQ 6 — Which **terms** are associated with at least **100** artists?
Return `term`, `n` ordered by `n` desc.

- **Option A (subquery):** group in a subquery then filter outer query
- **Option B (CTE):** name the grouped result and filter in the final select

In [ ]:
# ▶️ Option A — Subquery
from sqlalchemy import text
sql = """
SELECT term, n
FROM (
  SELECT at.term AS term, COUNT(DISTINCT at.artist_id) AS n
  FROM artist_term AS at
  GROUP BY at.term
) AS grouped
WHERE grouped.n >= 100
ORDER BY grouped.n DESC;
"""
with engine.connect() as conn:
    df_terms_100_subq = pd.read_sql_query(text(sql), conn)

df_terms_100_subq.head(10)

In [ ]:
# ▶️ Option B — CTE
from sqlalchemy import text
sql = """
WITH term_counts AS (
  SELECT at.term AS term, COUNT(DISTINCT at.artist_id) AS n
  FROM artist_term AS at
  GROUP BY at.term
)
SELECT term, n
FROM term_counts
WHERE n >= 100
ORDER BY n DESC;
"""
with engine.connect() as conn:
    df_terms_100_cte = pd.read_sql_query(text(sql), conn)

df_terms_100_cte.head(10)

### RQ 7 (independent) — Who shows up if we combine **rock-term artists** and **metal-mbtag artists**?
Return distinct `artist_id` for the union of:
- artists with term `'rock'` (from `artist_term`)
- artists with mbtag `'metal'` (from `artist_mbtag`)

Compare `UNION` vs `UNION ALL` (how many rows vs distinct rows?). Limit to 20 IDs, ascending.

In [ ]:
# 🔎 Your turn — UNION vs UNION ALL
sql = """
-- write your UNION query here
"""
with engine.connect() as conn:
    df_union = pd.read_sql_query(sql, conn)

df_union.head(20)

## 5) Quick pandas tally (optional)

Make a small bar chart of **RQ 1** (top terms) or **RQ 3** (top mbtags).

In [ ]:
import matplotlib.pyplot as plt

try:
    to_plot = df_top_terms.copy().head(10) if 'df_top_terms' in globals() else df_top_mbtags.copy().head(10)
    to_plot.plot(kind="bar", x=to_plot.columns[0], y=to_plot.columns[1], legend=False, title="Top categories by # of artists")
    plt.xlabel(to_plot.columns[0]); plt.ylabel("# artists")
    plt.show()
except Exception as e:
    print("Nothing to plot yet — complete RQ 1 or RQ 3 first.")

## 6) Brief interpretations

Write **1–2 sentences** for each (plain English):

- **RQ 1 or RQ 3:** What stands out in the top categories? Any surprises or limitations (e.g., naming inconsistencies, case sensitivity)?
- **RQ 7:** Did `UNION` vs `UNION ALL` change your counts? Why?

## Troubleshooting

- **`Access denied ...` (1045)** → Check your read‑only username/password; ask the instructor if needed.
- **Timeout / can’t connect (2003)** → Network or RDS security group issue. Try campus Wi‑Fi or VPN; ask the instructor to verify SG rules.
- **`ssl ca certificate` errors** → Re‑download or upload the correct `global-bundle.pem` and ensure `connect_args={"ssl_ca": "global-bundle.pem"}`.
- **`Table doesn't exist`** → Confirm `DB_NAME` and that the schema has `artists`, `terms`, `mbtags`, `artist_term`, `artist_mbtag`.

## Submit

- **Runtime → Restart and run all** to confirm a clean run
- **File → Download .ipynb**, rename to `Notebook3_LastFirst.ipynb`, and submit per syllabus